## Method-1

In [1]:
import os
import shutil
from sklearn.model_selection import train_test_split
import random

In [2]:
curr_path = os.getcwd()
print(curr_path)

D:\Masters\OVGU\VII_Semester\Thesis\jupyter_notebooks


In [3]:
# Directory paths
input_dir = os.path.join(curr_path, 'Data/Dataset/Full Dataset/tiles')
mask_dir = os.path.join(curr_path, 'Data/Dataset/Full Dataset/masks')

In [4]:
# List all input and mask files
input_files = sorted(os.listdir(input_dir))
mask_files = sorted(os.listdir(mask_dir))

print(len(input_files))
print(len(mask_files))

8021
8021


In [6]:
# Extract the DOP index from the filenames (input_0_0_0 and mask_0_0_0)
dop_indices = set([int(f.split('_')[1]) for f in input_files])
print(dop_indices)

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27}


In [7]:
# Split DOP indices into train, val, test (e.g., 70% train, 15% val, 15% test)
dop_indices = list(dop_indices)
train_dops, test_val_dops = train_test_split(dop_indices, test_size=0.3, random_state=42)
val_dops, test_dops = train_test_split(test_val_dops, test_size=0.5, random_state=42)

In [8]:
print(train_dops)
print(val_dops)
print(test_dops)

[25, 2, 5, 6, 3, 16, 23, 4, 26, 24, 19, 27, 21, 8, 11, 15, 20, 7]
[10, 1, 22, 17]
[18, 14, 12, 9, 13]


In [9]:
# Function to copy files to their respective folders
def copy_files(files, dest_dir):
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)
    for file in files:
        shutil.copy(file, dest_dir)

In [5]:
# Create directories for train, val, test
train_input_dir = os.path.join(curr_path, 'Data\\Dataset\\Final_Dataset_1\\train\\tiles')
train_mask_dir = os.path.join(curr_path, 'Data\\Dataset\\Final_Dataset_1\\train\\masks')
val_input_dir = os.path.join(curr_path, 'Data\\Dataset\\Final_Dataset_1\\val\\tiles')
val_mask_dir = os.path.join(curr_path, 'Data\\Dataset\\Final_Dataset_1\\val\\masks')
test_input_dir = os.path.join(curr_path, 'Data\\Dataset\\Final_Dataset_1\\test\\tiles')
test_mask_dir = os.path.join(curr_path, 'Data\\Dataset\\Final_Dataset_1\\test\\masks')

In [11]:
# Separate files by DOP index
train_input_files = [f for f in input_files if int(f.split('_')[1]) in train_dops]
train_mask_files = [f for f in mask_files if int(f.split('_')[1]) in train_dops]

val_input_files = [f for f in input_files if int(f.split('_')[1]) in val_dops]
val_mask_files = [f for f in mask_files if int(f.split('_')[1]) in val_dops]

test_input_files = [f for f in input_files if int(f.split('_')[1]) in test_dops]
test_mask_files = [f for f in mask_files if int(f.split('_')[1]) in test_dops]

In [12]:
# Copy files to the respective directories
copy_files([os.path.join(input_dir, f) for f in train_input_files], train_input_dir)
copy_files([os.path.join(mask_dir, f) for f in train_mask_files], train_mask_dir)

copy_files([os.path.join(input_dir, f) for f in val_input_files], val_input_dir)
copy_files([os.path.join(mask_dir, f) for f in val_mask_files], val_mask_dir)

copy_files([os.path.join(input_dir, f) for f in test_input_files], test_input_dir)
copy_files([os.path.join(mask_dir, f) for f in test_mask_files], test_mask_dir)


## Method-2

In [6]:
# Group tiles by DOP index (the first number in the filename)
tile_groups = {}
for f in input_files:
    dop_index = int(f.split('_')[1])
    if dop_index not in tile_groups:
        tile_groups[dop_index] = []
    tile_groups[dop_index].append(f)

In [7]:
# Calculate the total number of tiles
total_tiles = len(input_files)

# Define the split ratios
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# Calculate how many tiles we need for each split
train_tile_count = int(total_tiles * train_ratio)
val_tile_count = int(total_tiles * val_ratio)
test_tile_count = total_tiles - train_tile_count - val_tile_count  # remaining tiles for test

In [8]:
print(train_tile_count)
print(val_tile_count)
print(test_tile_count)

5614
1203
1204


In [9]:
# Function to copy files to their respective folders
def copy_files(files, input_dir, mask_dir, dest_input_dir, dest_mask_dir):
    # Create destination directories if they don't exist
    if not os.path.exists(dest_input_dir):
        os.makedirs(dest_input_dir)
    if not os.path.exists(dest_mask_dir):
        os.makedirs(dest_mask_dir)
    
    for file in files:
        # Copy the input file to the destination input directory
        shutil.copy(os.path.join(input_dir, file), dest_input_dir)
        
        # Find and copy the corresponding mask file to the destination mask directory
        mask_file = file.replace('input', 'mask')  # Assuming the file naming pattern
        shutil.copy(os.path.join(mask_dir, mask_file), dest_mask_dir)


In [10]:
# Select DOPs for validation and test sets while ensuring no overlap
all_dop_indices = list(tile_groups.keys())
random.shuffle(all_dop_indices)

val_dop_indices = []
test_dop_indices = []

val_tiles, test_tiles, train_tiles = [], [], []

In [11]:
# Select DOPs for validation set
for dop_index in all_dop_indices:
    val_dop_indices.append(dop_index)
    val_tiles.extend(tile_groups[dop_index])
    if len(val_tiles) >= val_tile_count:
        break

In [12]:
# Select DOPs for test set
for dop_index in all_dop_indices:
    if dop_index not in val_dop_indices:
        test_dop_indices.append(dop_index)
        test_tiles.extend(tile_groups[dop_index])
        if len(test_tiles) >= test_tile_count:
            break

In [13]:
# Assign remaining DOPs to training set
for dop_index in all_dop_indices:
    if dop_index not in val_dop_indices and dop_index not in test_dop_indices:
        train_tiles.extend(tile_groups[dop_index])

In [14]:
# Ensure train set has enough tiles (may have slight imbalance due to tile distribution)
while len(train_tiles) < train_tile_count and test_tiles:
    train_tiles.append(test_tiles.pop())

In [15]:
# Copy the selected files to their respective directories
copy_files(train_tiles, input_dir, mask_dir, train_input_dir, train_mask_dir)
copy_files(val_tiles, input_dir, mask_dir, val_input_dir, val_mask_dir)
copy_files(test_tiles, input_dir, mask_dir, test_input_dir, test_mask_dir)

print(f"Train tiles: {len(train_tiles)}, Validation tiles: {len(val_tiles)}, Test tiles: {len(test_tiles)}")

Train tiles: 5614, Validation tiles: 1545, Test tiles: 862
